Мы определяем следующий пул компаний для который будем считать Information Ratio:
* S&P 500 (^GSPC) - наш бенчмарк
* FORD (F)
* Amazon (AMZN)
* Tesla (TSLA)
* Apple (AAPL)
* Yandex (YNDX)
* NIVIDA(NVDA)
* Airbnb (ABNB)
* CocaCola Consolidated (COKE)
* Netflix (NFLX)


Сначала необходимо определить откуда мы будем брать информацию. Наша команда выбрала сайт Yahoo.com для импорта данных о ценных бумагах компаний, представленных выше.

In [279]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()
import datetime as dt
import plotly.graph_objects as go
import plotly.express as px

In [280]:
def get_data(name:str):
    '''
    Эта функция импортирует данные по названию компании
    '''
    data = pdr.get_data_yahoo([name], start='2020-12-10', end='2023-11-01')['Adj Close'].reset_index()
    data = pd.DataFrame(data)
    data.rename(columns={'Adj Close':  f'{name}'}, inplace=True)
    return data

short_names = ['^GSPC','F','AMZN','TSLA', 'AAPL', 'YNDX', 'NVDA', 'ABNB', 'COKE', 'NFLX']
data = get_data(short_names[0])
for i in short_names:
    if i != '^GSPC':
        data = pd.merge(data, get_data(i), on = 'Date', how = 'inner')
data


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
0,2020-12-10,3668.100098,7.952765,155.074493,209.023331,121.260780,70.680000,129.442139,144.710007,257.259735,501.089996
1,2020-12-11,3663.459961,7.865563,155.820999,203.330002,120.444115,70.279999,129.851303,139.250000,256.806305,503.220001
2,2020-12-14,3647.489990,7.769641,157.848495,213.276672,119.824226,68.230003,132.799881,130.000000,259.043762,522.419983
3,2020-12-15,3694.620117,7.978924,158.255997,211.083328,125.826263,68.889999,133.316269,124.800003,262.119080,519.780029
4,2020-12-16,3701.169922,7.883003,162.048004,207.589996,125.757378,70.800003,132.138840,137.990005,263.252594,524.830017
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,4186.770020,11.362824,121.389999,212.419998,171.100006,18.940001,417.790009,118.500000,641.820007,411.250000
723,2023-10-26,4137.229980,11.175742,119.570000,205.759995,166.889999,18.940001,403.260010,115.470001,639.960022,403.540009
724,2023-10-27,4117.370117,9.807083,127.739998,207.300003,168.220001,18.940001,405.000000,114.089996,634.080017,397.869995
725,2023-10-30,4166.819824,9.620000,132.710007,197.360001,170.289993,18.940001,411.609985,116.949997,640.890015,410.079987


Теперь необходимо посчитать Return для каждой компании

In [281]:
returns_daily = pd.DataFrame()
returns_daily['Date'] = data['Date']
for company in data.columns[1:]:
    returns_daily[company] = data[company].pct_change() * 100

returns_daily.iloc[1::]

,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
1,2020-12-11,-0.126500,-1.096494,0.481385,-2.723777,-0.673479,-0.565933,0.316098,-3.773068,-0.176254,0.425074
2,2020-12-14,-0.435926,-1.219518,1.301170,4.891885,-0.514669,-2.916897,2.270734,-6.642729,0.871263,3.815425
3,2020-12-15,1.292125,2.693604,0.258160,-1.028403,5.009035,0.967311,0.388847,-3.999998,1.187181,-0.505332
4,2020-12-16,0.177280,-1.202180,2.396122,-1.654954,-0.054747,2.772541,-0.883185,10.568912,0.432443,0.971562
5,2020-12-17,0.575765,0.442473,-0.150575,5.319782,0.696355,-0.211867,0.745706,6.565691,0.561634,1.537642
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,-1.433963,1.316940,-5.577161,-1.893592,-1.349168,0.000000,-4.314865,-2.876815,-0.714130,-0.599427
723,2023-10-26,-1.183252,-1.646442,-1.499300,-3.135300,-2.460553,0.000000,-3.477823,-2.556961,-0.289799,-1.874770
724,2023-10-27,-0.480028,-12.246695,6.832816,0.748449,0.796933,0.000000,0.431481,-1.195120,-0.918808,-1.405068
725,2023-10-30,1.201002,-1.907634,3.890723,-4.794984,1.230527,0.000000,1.632095,2.506793,1.073997,3.068839


Теперь необходимо посчитать Return компаний от каждого года: 2020, 2021, 2022.

In [282]:
returns_2020 = returns_daily[(returns_daily['Date'] >= dt.datetime(2020,12,11)) & (returns_daily['Date'] < dt.datetime(2021,1,4))]
returns_2021 = returns_daily[(returns_daily['Date'] >dt.datetime(2020,12,31)) & (returns_daily['Date'] < dt.datetime(2022,1,1))]
returns_2022 = returns_daily[(returns_daily['Date'] >dt.datetime(2021,12,31)) & (returns_daily['Date'] < dt.datetime(2023,1,1))]
returns_2023 = returns_daily[(returns_daily['Date'] >dt.datetime(2022,12,31)) & (returns_daily['Date'] < dt.datetime(2024,1,1))]
returns_2023

,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
518,2023-01-03,-0.400055,0.429924,2.166666,-12.242249,-3.740472,0.0,-2.045994,-0.701753,-6.967759,0.023741
519,2023-01-04,0.753897,2.825348,-0.792356,5.124885,1.031431,0.0,3.031806,4.499411,1.959467,4.902523
520,2023-01-05,-1.164553,1.998326,-2.372559,-2.903910,-1.060466,0.0,-3.281584,-1.138415,-0.458859,0.093730
521,2023-01-06,2.284078,2.693883,3.561115,2.465109,3.679400,0.0,4.164033,0.923495,3.354908,1.888917
522,2023-01-09,-0.076763,0.874394,1.486987,5.934901,0.408881,0.0,5.175308,0.813377,-1.897993,-0.120417
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,-1.433963,1.316940,-5.577161,-1.893592,-1.349168,0.0,-4.314865,-2.876815,-0.714130,-0.599427
723,2023-10-26,-1.183252,-1.646442,-1.499300,-3.135300,-2.460553,0.0,-3.477823,-2.556961,-0.289799,-1.874770
724,2023-10-27,-0.480028,-12.246695,6.832816,0.748449,0.796933,0.0,0.431481,-1.195120,-0.918808,-1.405068
725,2023-10-30,1.201002,-1.907634,3.890723,-4.794984,1.230527,0.0,1.632095,2.506793,1.073997,3.068839


In [283]:
def returns(df):
  returns_for_period = dict()
  for company in df.columns[1:]:
    returns_for_period[company] = np.mean(df[company])
  return returns_for_period

ret_for_2020 = returns(returns_2020)
ret_for_2021 = returns(returns_2021)
ret_for_2022 = returns(returns_2022)
ret_for_2023 = returns(returns_2023)
all_years ={"2020": ret_for_2020, "2021":ret_for_2021, "2022": ret_for_2022, '2023':ret_for_2023}
all_years

{'2020': {'^GSPC': 0.17067781301150498,
  'F': -0.2543657707677235,
  'AMZN': 0.35813544074667003,
  'TSLA': 0.9041436189927525,
  'AAPL': 0.5470851720250485,
  'YNDX': -0.09686952591591742,
  'NVDA': 0.05078545217412435,
  'ABNB': 0.2145810232278453,
  'COKE': 0.14972564882808387,
  'NFLX': 0.558797075113324},
 '2021': {'^GSPC': 0.09795217403134876,
  'F': 0.37779533147558675,
  'AMZN': 0.02083247373169359,
  'TSLA': 0.21884275949165047,
  'AAPL': 0.13057323825653183,
  'YNDX': -0.03485992463988359,
  'NVDA': 0.362833177314684,
  'ABNB': 0.10520243602623691,
  'COKE': 0.35562478210375315,
  'NFLX': 0.06273466760982835},
 '2022': {'^GSPC': -0.0745441715572881,
  'F': -0.16998020178331674,
  'AMZN': -0.22317375231715073,
  'TSLA': -0.32930592009316806,
  'AAPL': -0.0969605545164926,
  'YNDX': -0.38950248263294807,
  'NVDA': -0.1990950889197472,
  'ABNB': -0.19209585415141225,
  'COKE': -0.024134110515469705,
  'NFLX': -0.17862986202537282},
 '2023': {'^GSPC': 0.045817922275342746,
  'F'

In [284]:
returns_for_all_per = returns(returns_daily)
returns_for_all_per

{'^GSPC': 0.024709085306788267,
 'F': 0.06401919841639772,
 'AMZN': 0.007397422857323051,
 'TSLA': 0.06496178037411644,
 'AAPL': 0.0631595379897443,
 'YNDX': -0.1486308505687927,
 'NVDA': 0.21417766397540505,
 'ABNB': 0.029498254135215667,
 'COKE': 0.15587412755391666,
 'NFLX': 0.025604054991031248}

Посчитаем Tracking Error для каждой компании за каждый год:

In [285]:
def tracking_error(df):
  tracking_errors = dict()
  diff_returns = pd.DataFrame()
  # Сначала необходимо посчитать разницу в Return бенчмарка и Return компаний
  for company in df.columns[2:]:
    diff_returns[f'{company}_diff'] = df[company] - df['^GSPC']
  for company in diff_returns.columns[2:]:
    tracking_errors[company[:4]] = diff_returns[company].std()
  return tracking_errors

err_2020 = tracking_error(returns_2020)
err_2021 = tracking_error(returns_2021)
err_2022 = tracking_error(returns_2022)
err_2023 = tracking_error(returns_2023)
all_err = {"2020": err_2020, "2021":err_2021, "2022": err_2022, '2023':err_2023}
all_err

{'2020': {'TSLA': 3.525953430088085,
  'AAPL': 1.7096652581652794,
  'YNDX': 1.5846493185763348,
  'NVDA': 1.2825299227102671,
  'ABNB': 5.087712293547973,
  'COKE': 1.1371644384383681,
  'NFLX': 1.802931964482678},
 '2021': {'TSLA': 3.166323511664273,
  'AAPL': 1.1794930625382039,
  'YNDX': 1.8314219674848011,
  'NVDA': 2.444854649805731,
  'ABNB': 3.219386377767941,
  'COKE': 1.9948075576441768,
  'NFLX': 1.83523142207129},
 '2022': {'TSLA': 3.440172641698223,
  'AAPL': 1.1444806880568952,
  'YNDX': 3.631567476742975,
  'NVDA': 2.8154906839498874,
  'ABNB': 2.9593126184774565,
  'COKE': 2.744873384816823,
  'NFLX': 3.7560907496940827},
 '2023': {'TSLA': 3.1675909766298087,
  'AAPL': 0.9222161519734682,
  'YNDX': 0.8480584961941273,
  'NVDA': 2.8564648963830206,
  'ABNB': 2.3789097124906635,
  'COKE': 2.0952313150664765,
  'NFLX': 2.2613936108559773}}

In [286]:
data_error = pd.DataFrame(all_err).T.reset_index().rename(columns={'index': 'Годы'})

data_melted = data_error.melt(id_vars=['Годы'], var_name='Компания', value_name='Tracking Error')
data_melted
fig = go.Figure()

for company in data_melted['Компания'].unique():
  fig.add_trace(go.Violin(x=data_melted['Компания'][data_melted['Компания'] == company], y=data_melted['Tracking Error'][data_melted['Компания'] == company], name=company,
                            box_visible=True,
                            meanline_visible=True))
  
fig.update_layout(title = {
        "text": "Распределение Tracking Error каждой компании по годам",
        "x": 0.5},
        legend=dict(
        title="Компания",
        yanchor="bottom",
        y=0.3,
        xanchor="right",
        x=1.07
    )      
)
fig.show()

Сейчас необходимо посчитать Tracking Error для каждой компании за весь период:

In [287]:
tracking_errors = tracking_error(returns_daily)
tracking_errors

{'TSLA': 3.2722920907553306,
 'AAPL': 1.1109351151275884,
 'YNDX': 2.4438976580785736,
 'NVDA': 2.6904174922007784,
 'ABNB': 2.9507293859034074,
 'COKE': 2.2950410210617296,
 'NFLX': 2.751275211557334}

Теперь считаем сам Information Ratio для каждой компании по годам

In [288]:
def IR(df):
  information_ratio = dict()
  for company in tracking_errors.keys():
    information_ratio[company] = (df[company] - df['^GSPC']) / tracking_errors[company]
  return information_ratio

ratio_for_2020 = pd.DataFrame(IR(all_years['2020']), index = [0])
ratio_for_2021 = pd.DataFrame(IR(all_years['2021']), index = [0])
ratio_for_2022 = pd.DataFrame(IR(all_years['2022']), index = [0])
ratio_for_2023 = pd.DataFrame(IR(all_years['2023']), index = [0])
print(f'for 2020:\n {ratio_for_2020}\n')
print(f'for 2021:\n {ratio_for_2021}\n')
print(f'for 2022:\n {ratio_for_2022}\n')
print(f'for 2023:\n {ratio_for_2023}\n')

for 2020:
        TSLA     AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0  0.224144  0.33882 -0.109476 -0.044563  0.014879 -0.009129  0.141069

for 2021:
        TSLA      AAPL      YNDX      NVDA      ABNB      COKE    NFLX
0  0.036944  0.029364 -0.054344  0.098453  0.002457  0.112274 -0.0128

for 2022:
        TSLA      AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0 -0.077854 -0.020178 -0.128875 -0.046294 -0.039838  0.021965 -0.037832

for 2023:
        TSLA      AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0  0.076671  0.086359 -0.018748  0.184502  0.049523  0.037386  0.052549



Теперь считаем сам Information Ratio для каждой компании за весь период

In [289]:
ratio_for_all = IR(returns_for_all_per)
ratio_for_all

{'TSLA': 0.01230107030513795,
 'AAPL': 0.034610889654469235,
 'YNDX': -0.07092765742566455,
 'NVDA': 0.07042348602693267,
 'ABNB': 0.0016230457632973104,
 'COKE': 0.0571515023232347,
 'NFLX': 0.0003252926790033453}

Графики являются интерактивными. По нажатию на компанию в легенде, можно убрать её кривую.

In [290]:
years = [2020, 2021, 2022, 2023]
data = pd.concat([ratio_for_2020, ratio_for_2021, ratio_for_2022, ratio_for_2023], ignore_index=True)
data
fig = go.Figure()
for company in data.columns:
    fig.add_trace(go.Scatter(x=years, y=data[company], name=company))
fig.update_layout(title = {
    "text": "Распределение Tracking Error каждой компании по годам",
    "x": 0.5},
    legend=dict(
    title="Компания",
    yanchor="bottom",
    y=0.3,
    xanchor="right",
    x=1.07),
    xaxis = dict(dtick = 1)      
)
fig.show()


In [292]:
fig = px.imshow(data,
                labels=dict(x="Компании", y="Годы", color="Изменение IR"),
                x=data.columns,
                y=years,
                color_continuous_scale='purd')

fig.update_layout(
    title='Тепловая карта изменений IR компаний за 2020-2023 годы',
    yaxis = dict(dtick = 1),
    xaxis_nticks=36)

fig.show()